## Text Generation using GPT2 model

### Project: Ads generation from product description

In [1]:
!pip install datasets transformers accelerate


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import torch
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
from datasets import Dataset

2025-01-27 17:13:30.572121: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-27 17:13:30.585937: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737987210.606152    8277 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737987210.610769    8277 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-27 17:13:30.627565: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
model_name = "gpt2-large"  #or just gpt2
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = TFGPT2LMHeadModel.from_pretrained(model_name, pad_token_id=tokenizer.eos_token_id)

I0000 00:00:1737987217.267517    8277 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3539 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [7]:
def generate_advertisement(product_description, max_length=100):
    input_text = "Product: " + product_description + "\nAdvertisement:"

    # Encode input text into ids- tokenization
    input_ids = tokenizer.encode(input_text, return_tensors="tf")

    # Generate text
    output = model.generate(input_ids, max_length=max_length)

    # decode the ids back into text
    generated_ads = []
    for sample in output:
        generated_ad = tokenizer.decode(sample, skip_special_tokens=True)
        generated_ads.append(generated_ad)

    return generated_ads

In [8]:
product_description = "Introducing our latest smartphone model, with a powerful processor and stunning camera features."

# Generate advertisements
generated_ads = generate_advertisement(product_description, max_length=150)

### Predicted response

In [9]:
generated_ads

['Product: Introducing our latest smartphone model, with a powerful processor and stunning camera features.\nAdvertisement:\nThe new model is the first to feature a 5.5-inch display, a Qualcomm Snapdragon 810 processor, 4GB of RAM, and a 13MP rear camera with a f/2.0 aperture. The phone also features a fingerprint sensor, a 3,000mAh battery, and a 3,000mAh removable battery. The phone will be available in two colors: black and white.\nThe phone will be available in China starting on September 1st, and will be priced at 2,499 yuan (about $400).']

## Using Greedy approach-

With Greedy search, the word with the highest probability is predicted as the next word:

### $w_t=argmax_wP(w|w_1:_t-_1)$

Beam search is essentially Greedy Search but the model tracks and keeps num_beams of hypotheses at each time step, so the model is able to compare alternative paths as it generates text. We can also include a n-gram penalty by setting no_repeat_ngram_size = 3 which ensures that no 3-grams appear thrice

In [10]:
def generate_advertisement_greedy(product_description):
    input_text = "Product: " + product_description + "\nAdvertisement:"

    # Encode input text- use number of beams, ngram size
    input_ids = tokenizer.encode(input_text, num_beams = 7,no_repeat_ngram_size=3,num_return_sequences=5,early_stopping = True,return_tensors="tf")

    # Generate text
    output = model.generate(input_ids, max_length=150)

    # decode the ids back into text
    generated_ads = []
    for sample in output:
        generated_ad = tokenizer.decode(sample, skip_special_tokens=True)
        generated_ads.append(generated_ad)

    return generated_ads

In [11]:
generated_ads_greedy = generate_advertisement_greedy(product_description)

Keyword arguments {'num_beams': 7, 'no_repeat_ngram_size': 3, 'num_return_sequences': 5, 'early_stopping': True} not recognized.


In [12]:
generated_ads_greedy

['Product: Introducing our latest smartphone model, with a powerful processor and stunning camera features.\nAdvertisement:\nThe new model is the first to feature a 5.5-inch display, a Qualcomm Snapdragon 810 processor, 4GB of RAM, and a 13MP rear camera with a f/2.0 aperture. The phone also features a fingerprint sensor, a 3,000mAh battery, and a 3,000mAh removable battery. The phone will be available in two colors: black and white.\nThe phone will be available in China starting on September 1st, and will be priced at 2,499 yuan (about $400).']